# Передача аргументов командной строки

Параметры запуска, задаваемые через командную строку, чаще всего используют консольные программы, хотя программы с графическим интерфейсом тоже не брезгуют этой возможностью.

Разберем несколько способов разобрать аргументы командной строки.

## Переменная argv

`sys.argv` содержит список параметров, переданных программе через командную строку, причем нулевой элемент списка - это имя скрипта.

Этот способ не очень удобный, поэтому используется редко или только в совсем несложных проектах.

In [ ]:
import sys

for param in sys.argv:
    print (param)

## Библиотека argparse

Стандартная библиотека **argparse** предназначена для облегчения разбора командной строки. На нее можно возложить проверку переданных параметров: их количество и обозначения, а уже после того, как эта проверка будет выполнена автоматически, использовать полученные параметры в логике своей программы.

Основа работы с командной строкой в библиотеке argparse является класс **ArgumentParser**. У его конструктора и методов довольно много параметров, все их рассматривать не будем, поэтому в дальнейшем рассмотрим работу этого класса на примерах, попутно обсуждая различные параметры.

Простейший принцип работы с argparse следующий:

- Создаем экземпляр класса ArgumentParser.
- Добавляем в него информацию об ожидаемых параметрах с помощью метода add_argument (по одному вызову на каждый параметр).
- Разбираем командную строку помощью метода parse_args, передавая ему полученные параметры командной строки (кроме нулевого элемента списка sys.argv).
- Начинаем использовать полученные параметры.

In [ ]:
# Ожидаемый вызов программы:
# python coolprogram.py [Имя]
# где [Имя] является необязательным параметром

import sys
import argparse


def createParser():
    
    # экземпляр класса ArgumentParser с параметрами по умолчанию
    parser = argparse.ArgumentParser()
    
    # Параметр
    parser.add_argument ('name', nargs='?')
    # Такой параметр будет считаться позиционным, 
    # т.е. он должен стоять именно на этом месте 
    # и у него не будет никаких предварительных обозначений.
    
    # nargs - сколько аргументов ожидаем,
    # может принимать значение '?', '+', '*' или число
 
    return parser
 
if __name__ == '__main__':
    parser = createParser()
    namespace = parser.parse_args()

    # при запуске: python coolprogram.py Вася
    # namespace(name='Вася')

    # при запуске: python coolprogram.py
    # nmespace(name=None)
    
    if namespace.name:
        print ("Привет, {}!".format (namespace.name) )
    else:
        print ("Привет, мир!")

In [ ]:
# Параметры со значением по умолчанию

import sys
import argparse
 

def createParser():
    parser = argparse.ArgumentParser()
    # параметр со значением по умолчанию 'мир'
    parser.add_argument ('name', nargs='?', default='мир')
 
    return parser
 
if __name__ == '__main__': 
    parser = createParser()
    namespace = parser.parse_args (sys.argv[1:])

    print ("Привет, {}!".format (namespace.name) )

In [ ]:
# Именованные параметры

import sys
import argparse
 

def createParser():
    parser = argparse.ArgumentParser()
    # именованный параметр
    # должен передаваться после параметра --name или -n.
    parser.add_argument ('-n', '--name', default='мир')
 
    return parser
 
if __name__ == '__main__':
    parser = createParser()
    namespace = parser.parse_args(sys.argv[1:])
 
    print ("Привет, {}!".format (namespace.name) )

# Все именованные параметры считаются необязательными!
# Чтобы именованный параметр стал обязательным,
# можно добавить 'required=True'

In [ ]:
# Список разрешенных параметров

import sys
import argparse
 
def createParser ():
    parser = argparse.ArgumentParser()
    parser.add_argument ('-n', '--name', choices=['Вася', 'Оля', 'Петя'], default='Оля')
 
    return parser

if __name__ == '__main__':
    parser = createParser()
    namespace = parser.parse_args(sys.argv[1:])
 
    print ("Привет, {}!".format (namespace.name) )

In [ ]:
# Указание типов параметров

import sys
import argparse
 
def createParser ():
    parser = argparse.ArgumentParser()
    parser.add_argument ('-c', '--count', default=1, type=int) 
    # в качестве значения параметра type мы передали не строку, 
    # а стандартную функцию преобразования в целое число
 
    return parser
 
if __name__ == '__main__':
    parser = createParser()
    namespace = parser.parse_args(sys.argv[1:])
 
    for _ in range (namespace.count):
        print ("Привет, мир!")

In [ ]:
# Так как в type передается функция, можно таким способом проверить файл

import sys
import argparse
 
def createParser ():
    parser = argparse.ArgumentParser()
    parser.add_argument ('-n', '--name', type=open)
    
    # более изящное решение:
    parser.add_argument ('-n', '--name', type=argparse.FileType())
    # функция argparse.FileType, предназначенной для безопасной попытки открытия файла
    
    return parser

if __name__ == '__main__':
    parser = createParser()
    namespace = parser.parse_args(sys.argv[1:])
 
    text = namespace.name.read()
 
    print (text)

## Библиотека click

Click решает ту же проблему, что и argparse, но немного иначе. Он использует декораторы, поэтому команды должны быть функциями, которые можно обернуть этими декораторами.

Он принципиально отличается от argparse количеством функционала и подходом к описанию команд и параметров через декораторы, а саму логику предлагается выделять в отдельные функции вместо большого main. Авторы утверждают, что у Click много настроек, но стандартных параметров должно хватить. Среди фич подчёркиваются вложенные команды и их ленивая подгрузка. 

In [ ]:
# декоратор @click.command() превращает функцию в команду, 
# которая является главной точкой входа нашего скрипта.

import click
import datetime

@click.group()
def cli():
    pass

@click.command()
# option - опция, необязательный. argument - параметр, обязательный
@click.option('--date', default='now', help='The date format "yyyy-mm-dd"')
def get_weekday(date):
    if date == 'now':
        date = datetime.datetime.utcnow()
    else:
        date = datetime.datetime.strptime(date, '%Y-%m-%d')

    click.echo(date.strftime('%A'))

@click.command()
@click.option('--date1', help='The date format "yyyy-mm-dd"')
@click.option('--date2', help='The date format "yyyy-mm-dd"')
def delta_day(date1, date2):
    date1 = datetime.datetime.strptime(date1, '%Y-%m-%d')
    date2 = datetime.datetime.strptime(date2, '%Y-%m-%d')
    delta = date1 - date2 if date1 > date2 else date2 - date1
    click.echo(delta.days)

cli.add_command(get_weekday)
cli.add_command(delta_day)

if __name__ == '__main__':
    cli()

Флаг `--help` выведет автоматически сгенерированную документацию.

Добавив справочный текст в декоратор `@click.option(... help='...')`, мы добавим описание для опций и параметров.

Добавить документацию для всей click-команды можно, добавив строку документации в основную функцию:

```
def cli():
    '''
    This program make
    cool things
    '''
    pass
```